# Imports

In [3]:
import numpy as np
import random

# Initialization functions

In [4]:
#generates a 1D array (solution) of zeros and ones with probability p
#
#input: random 1d array with values in [0,1)
#output: converts values to 1 if element < p
def init_1D_p(x,p=0.5):
    for i in range(len(x)):
        if x[i]<p:
            x[i] = 1
        else:
            x[i]=0
    return x

In [5]:
#initializes k 1D solutions of length n
#
#output: row i is solution i
def init_solns_1D(k,n,p=0.5):
    solns = np.zeros((k,n))
    for i in range(k):
        solns[i] = init_1D_p(np.random.rand(n),p)
    return solns

# Genetic functions

In [6]:
#mates x1 and x2 and randomizes elements from child < p
def mate_1D(x1,x2):
    rand_mate=np.random.rand(len(x1))
    result= np.zeros(len(x1))
    for i in range(len(x1)):
        if rand_mate[i] < 0.5:
            result[i] = x1[i]
        else:
            result[i] = x2[i]          
    return result
    

In [7]:
#applies random mutations to elements of x with probability p
def random_mutation_1D(x,p=.1):
    import random
    rand_mate=np.random.rand(len(x))
    for i in range(len(x)):
        if rand_mate[i]<p:
            x[i] = random.sample([0,1],1)[0]
    return x

In [8]:
#mutates a set of solutions
def mutate(solns):
    for i,soln in enumerate(solns):
        solns[i]=random_mutation_1D(soln)
    return solns

# Constraints

In [9]:
#x: solutions with each row as a solution
#c: coefficients so c_i is constraint coefficient for x_i
#ub: the upperbound 
#
#return: boolean value; true if contraint satisfied
def Constraint1_UpperBound(x):
    c = np.array([1,2,3])
    ub=4
    return np.dot(x,c)<ub
    #return sum(x*c)<ub

# Objective Function

In [10]:
#x: solutions with each row as a solution
#o: coefficients of objective function
def Linear_Objective(x):
    o = np.array([1,2,3])
    return np.dot(x,o)

In [11]:
#create function to store solutions
def updateSolnToResult(soln,dictionary,objective_function):
    dictionary[str(soln)]=objective_function(soln)
    return dictionary

# Evalution and Simulations

In [12]:
#set some hyperparameters for what we're working with and initialize solutions
num_steps =100 #number of epochs
num_solns = 10 #number of solutions we are working with
solnToResult = {}
solns = init_solns_1D(num_solns,3)

In [13]:
#booleans that are true if the index contains a working solution
def getWorkingSolutionIndices(constraint_results):
    #results = np.array(sum(constraint_results))
    results = []
    for i in range(len(constraint_results)):
        if constraint_results[i] == True:
            results.append(i)
    return np.array(results)

In [14]:
#implementation that only cares about satisfying constraint
#doesn't care about obj
for _ in range(num_steps):
    new_solns = np.zeros((num_solns,3))
    obj_results = Linear_Objective(solns)
    c1_results = Constraint1_UpperBound(solns) #check constraints
    #for now, just mate are replace non-working solutions with random ones

    #get working solution indices
    working_soln_ind = getWorkingSolutionIndices(c1_results)


    #replace failed solutions with randmized ones
    for i in range(num_solns):
        if c1_results[i] == True:
            if(len(working_soln_ind)>=2): #check if mating is possible
                #pick random solution to mate with (including itself)
                mate_index = random.sample(working_soln_ind,1)[0]
                offspring = mate_1D(solns[i],solns[mate_index])
            else:
                offspring = solns[i]
        else: #replace it with a some random solutions
            offspring = init_1D_p(np.random.rand(3))
        new_solns[i] = offspring

    solns = new_solns #reset the solution set
print solns      

[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]]


Although all the solutions will converge to meet our constraint, it is not the optimal solution. Time for mating to understand attraction. 

In [17]:
#returns a probability distribution with weighted values 
#corresponding to objective solutions that satisfy constraints
def getPDF(solns,working_soln_ind,obj_results):
    total = sum(obj_results[working_soln_ind])
    prob = np.zeros(len(solns))
    for i in working_soln_ind:
        prob[i]=obj_results[i]/total
    return prob
    

In [18]:
#input:a probability mass function
#output: corresponding cdf
def getCDF(pmf):
    cdf = np.zeros(len(pmf))
    running_total = 0
    for i,p in enumerate(pmf):
        running_total += p
        cdf[i] = running_total
    return cdf
        

In [19]:
#samples pmf once and returns the index of the random sample
def sample_p_dist(pmf):
    cdf = getCDF(pmf)
    rand_num = random.random()
    return np.argmax(cdf==cdf[np.argmax(cdf>rand_num)])

In [21]:
#reinitialize the solns
solnToResult={}
solns = init_solns_1D(num_solns,3)

In [22]:
#implementation that cares about satisfying constraint and maximizing our function
for _ in range(num_steps):
    new_solns = np.zeros((num_solns,3))
    obj_results = Linear_Objective(solns) #get objective value for each solution
    c1_results = Constraint1_UpperBound(solns) #check constraints
    
    #get working solution indices
    working_soln_ind = getWorkingSolutionIndices(c1_results)
    
    #get pmf of distribution to sample from
    prob = getPDF(solns,working_soln_ind,obj_results)

    #replace failed solutions with randmized ones
    for i in range(num_solns):
        if c1_results[i] == True:
            updateSolnToResult(solns[i],solnToResult,Linear_Objective)
            if(len(working_soln_ind)>=2): #check if mating is possible
                #pick random solution to mate with according to the pmf from weighted obj results
                mate_index=sample_p_dist(prob)
                offspring = mate_1D(solns[i],solns[mate_index])
            else:
                offspring = solns[i]
        else: #replace it with a some working solutions
            if(len(working_soln_ind)>=2): #check if mating is possible
                mate_index1,mate_index2 = sample_p_dist(prob),sample_p_dist(prob)
                offspring = mate_1D(solns[mate_index1],solns[mate_index2])
            else: #replace it with a some random solutions
                offspring = init_1D_p(np.random.rand(3))
        new_solns[i] = offspring
        
    solns = mutate(new_solns) #reset the solution set and mutate
    
print solns

[[ 1.  1.  0.]
 [ 1.  1.  1.]
 [ 1.  1.  0.]
 [ 1.  1.  0.]
 [ 1.  1.  0.]
 [ 1.  1.  0.]
 [ 1.  1.  0.]
 [ 1.  1.  0.]
 [ 1.  1.  0.]
 [ 0.  1.  0.]]


In [23]:
print solnToResult

{'[ 0.  0.  1.]': 3.0, '[ 1.  1.  0.]': 3.0, '[ 1.  0.  0.]': 1.0, '[ 0.  1.  0.]': 2.0, '[ 0.  0.  0.]': 0.0}
